In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
# Завантаження даних
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
# Нормалізація даних
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

**Базова Модель**

---


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras import optimizers
# Компіляція моделі
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )

In [ ]:
# # Тренування моделі
# history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_split=0.2)

from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/20
750/750 [==============================] - 13s 10ms/step - loss: 0.4540 - accuracy: 0.8440 - val_loss: 0.3796 - val_accuracy: 0.8538 - lr: 0.0010
Epoch 2/20
750/750 [==============================] - 7s 10ms/step - loss: 0.2968 - accuracy: 0.8955 - val_loss: 0.2580 - val_accuracy: 0.9049 - lr: 0.0010
Epoch 3/20
750/750 [==============================] - 7s 9ms/step - loss: 0.2522 - accuracy: 0.9110 - val_loss: 0.2358 - val_accuracy: 0.9141 - lr: 0.0010
Epoch 4/20
750/750 [==============================] - 7s 10ms/step - loss: 0.2245 - accuracy: 0.9202 - val_loss: 0.2110 - val_accuracy: 0.9233 - lr: 0.0010
Epoch 5/20
750/750 [==============================] - 7s 9ms/step - loss: 0.2022 - accuracy: 0.9257 - val_loss: 0.2323 - val_accuracy: 0.9168 - lr: 0.0010
Epoch 6/20
750/750 [==============================] - 7s 9ms/step - loss: 0.1874 - accuracy: 0.9320 - val_loss: 0.2293 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 7/20
750/750 [==============================] - 8s 10ms/step

In [ ]:
# Оцінка моделі
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")

313/313 [==============================] - 1s 3ms/step - loss: 0.2173 - accuracy: 0.9371
Test Accuracy: 93.71%


---

# **Якщо порівняти точність отриманої згорткової мережі (Conv2D) з точністю багатошарової мережі з попереднього завдання то можна прийти висновку що вона зросла на ~ 5%**

---



In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [2]:
# Завантаження та підготовка датасету
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
# Перетворення до формату з 3 каналами, що потрібно для VGG16
x_train = tf.image.grayscale_to_rgb(tf.expand_dims(x_train, axis=-1))
x_test = tf.image.grayscale_to_rgb(tf.expand_dims(x_test, axis=-1))

In [4]:
# Зміна розміру зображень до 32х32
x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])


In [5]:
# Конвертація типу даних до float32
x_train = tf.cast(x_train, tf.float32)
x_test = tf.cast(x_test, tf.float32)

In [6]:
# Масштабування пікселів до діапазону [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

In [7]:
# Категоріальна перекодування міток
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [8]:
# Завантаження передобученої моделі VGG16 без верхніх шарів
conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))

In [9]:
# Налаштування шарів для тренування
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == "block5_conv1":
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [10]:
# Створення нової моделі
modified_model = models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax"),
])

In [11]:
# Компіляція моделі
modified_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
# Використання зворотних викликів
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)


In [13]:
# Навчання моделі
history = modified_model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=[reduce_lr, early_stopping, model_checkpoint]
)

Epoch 1/20
750/750 [==============================] - ETA: 0s - loss: 0.5078 - accuracy: 0.8237

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


750/750 [==============================] - 24s 22ms/step - loss: 0.5078 - accuracy: 0.8237 - val_loss: 0.3257 - val_accuracy: 0.8818 - lr: 0.0010
Epoch 2/20
750/750 [==============================] - 16s 21ms/step - loss: 0.3490 - accuracy: 0.8810 - val_loss: 0.3041 - val_accuracy: 0.8912 - lr: 0.0010
Epoch 3/20
750/750 [==============================] - 16s 22ms/step - loss: 0.2968 - accuracy: 0.8964 - val_loss: 0.2953 - val_accuracy: 0.8962 - lr: 0.0010
Epoch 4/20
750/750 [==============================] - 17s 22ms/step - loss: 0.2814 - accuracy: 0.9011 - val_loss: 0.2836 - val_accuracy: 0.8999 - lr: 0.0010
Epoch 5/20
750/750 [==============================] - 17s 22ms/step - loss: 0.2641 - accuracy: 0.9081 - val_loss: 0.2859 - val_accuracy: 0.9000 - lr: 0.0010
Epoch 6/20
750/750 [==============================] - 16s 21ms/step - loss: 0.2565 - accuracy: 0.9104 - val_loss: 0.2879 - val_accuracy: 0.9007 - lr: 0.0010
Epoch 7/20
750/750 [==============================] - 16s 22ms/step -

In [14]:
# Завантаження найкращої моделі
modified_model.load_weights('best_model.h5')

In [15]:
# Оцінка моделі
test_loss, test_acc = modified_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")

313/313 [==============================] - 4s 12ms/step - loss: 0.3499 - accuracy: 0.9120
Test Accuracy: 91.20%


---

# **Якщо порівняти точність отриманої згорткової мережі (VGG16) з точністю багатошарової мережі з попереднього завдання то можна прийти висновку що вона зросла на ~ 3-4%**

---
